In [3]:
import chromadb
from config import CHROMA_DB_PATH, CHROMA_COLLECTION_NAME
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = client.get_or_create_collection(
    name=CHROMA_COLLECTION_NAME,
    metadata={"hnsw:space": "cosine"}#{"metric": "cosine"}
)

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
model=HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

In [ ]:
query="modi schemes for navsari"

In [23]:
query_embedding=model.embed_query(query)
results = collection.query(query_embeddings=[query_embedding], n_results=5)

In [6]:
from sentence_transformers import SentenceTransformer, CrossEncoder
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [24]:
results['documents']

[["SizeAbcSmallAbcMediumAbcLargeSavePrintCommentSynopsisPrime Minister Narendra Modi's visit to Vantara highlighted his commitment to animal welfare. He interacted with endangered species like lemurs, flamingos, otters, pythons, orangutans, and a giant hippopotamus, reflecting the sanctuary's success in providing safe habitats for these creatures.ET SpecialPM Narendra Modi's visit to Vantara reflected his shared dedication to animal welfare and harmonious coexistence. During the visit, he encountered several rare species. Some images, of the Prime Minister exploring Vantara's wildlife.PM Modi nurtures endangered lemursET SpecialFacing the looming threat of extinction, these rare creatures have found a safe sanctuary in Anant Ambani’s Vantara. Here, they rest peacefully on PM Modi’s lap, reflecting a bond of trust and care.A different shade of Orange: Flamingos walk gracefully around PM ModiET SpecialPM Modi was thrilled to witness the flamboyance of flamingos as they gracefully walked"

In [25]:
cross_inp = [[query, hit] for hit in results['documents'][0]]

In [26]:
cross_scores = reranker.predict(cross_inp)

In [31]:
import numpy as np
indexes = np.where(cross_scores > 0)[0]

In [32]:
indexes

array([0, 1, 2, 3, 4])

In [ ]:
hits = sorted(hits, key=lambda x: x["cross-encoder_score"], reverse=True)

In [27]:
cross_inp

[['modi visit to vantara',
  "SizeAbcSmallAbcMediumAbcLargeSavePrintCommentSynopsisPrime Minister Narendra Modi's visit to Vantara highlighted his commitment to animal welfare. He interacted with endangered species like lemurs, flamingos, otters, pythons, orangutans, and a giant hippopotamus, reflecting the sanctuary's success in providing safe habitats for these creatures.ET SpecialPM Narendra Modi's visit to Vantara reflected his shared dedication to animal welfare and harmonious coexistence. During the visit, he encountered several rare species. Some images, of the Prime Minister exploring Vantara's wildlife.PM Modi nurtures endangered lemursET SpecialFacing the looming threat of extinction, these rare creatures have found a safe sanctuary in Anant Ambani’s Vantara. Here, they rest peacefully on PM Modi’s lap, reflecting a bond of trust and care.A different shade of Orange: Flamingos walk gracefully around PM ModiET SpecialPM Modi was thrilled to witness the flamboyance of flamingos

In [28]:
cross_scores

array([9.501378 , 5.001235 , 4.377794 , 7.55329  , 6.7584667],
      dtype=float32)

In [ ]:
results['documents'][0]

[['to prevent rainwater from going to waste by conserving it where it falls. He praised the efforts of the women in Navsari, who have completed over 5,000 projects, including ponds, check dams, borewell recharge, and community soak pits, to save rainwater. He mentioned that hundreds of water conservation projects are still ongoing in Navsari, with the goal of constructing 1,000 percolation pits in a single day. Shri Modi acknowledged Navsari district as one of the leading districts in Gujarat for rainwater harvesting and water conservation, extending special congratulations to the mothers, sisters, and daughters of Navsari for their achievements in this area. “The strength of Gujarat’s women and their contributions are not limited to any one sector”, said Shri Modi, mentioning that 50% of seats in Gujarat’s panchayat elections were reserved for women. He remarked that when he was sent to Delhi as the Prime Minister, he brought the same experience and commitment to the nation. He noted'

In [5]:
results

{'ids': [['30', '20', '21', '40', '22']],
 'embeddings': None,
 'documents': [['to prevent rainwater from going to waste by conserving it where it falls. He praised the efforts of the women in Navsari, who have completed over 5,000 projects, including ponds, check dams, borewell recharge, and community soak pits, to save rainwater. He mentioned that hundreds of water conservation projects are still ongoing in Navsari, with the goal of constructing 1,000 percolation pits in a single day. Shri Modi acknowledged Navsari district as one of the leading districts in Gujarat for rainwater harvesting and water conservation, extending special congratulations to the mothers, sisters, and daughters of Navsari for their achievements in this area. “The strength of Gujarat’s women and their contributions are not limited to any one sector”, said Shri Modi, mentioning that 50% of seats in Gujarat’s panchayat elections were reserved for women. He remarked that when he was sent to Delhi as the Prime Min

In [29]:
help(reranker)

Help on CrossEncoder in module sentence_transformers.cross_encoder.CrossEncoder object:

class CrossEncoder(transformers.utils.hub.PushToHubMixin)
 |  CrossEncoder(model_name: 'str', num_labels: 'int' = None, max_length: 'int' = None, device: 'str | None' = None, automodel_args: 'dict' = None, tokenizer_args: 'dict' = None, config_args: 'dict' = None, cache_dir: 'str' = None, trust_remote_code: 'bool' = False, revision: 'str | None' = None, local_files_only: 'bool' = False, default_activation_function=None, classifier_dropout: 'float' = None) -> 'None'
 |  
 |  A CrossEncoder takes exactly two sentences / texts as input and either predicts
 |  a score or label for this sentence pair. It can for example predict the similarity of the sentence pair
 |  on a scale of 0 ... 1.
 |  
 |  It does not yield a sentence embedding and does not work for individual sentences.
 |  
 |  Args:
 |      model_name (str): A model name from Hugging Face Hub that can be loaded with AutoModel, or a path to a